# Project Grading

> Manage grading rubrics and grade posting on the groups grade.

In [ ]:
#| default_exp grading

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#| export
from CanvasGroupy.github import GitHubGroup
from CanvasGroupy.canvas import CanvasGroup
import github
import canvasapi
from ast import literal_eval

In [ ]:
#| export
#| hide
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

In [ ]:
#| export
class Grading:
    def __init__(self,
                 ghg=None, # authenticated GitHub object
                 cg=None, # authenticated canvas object
                ):
        self.ghg = ghg
        self.cg = cg

    def create_issue_from_md(self,
                              repo:github.Repository.Repository, # target repository to create issue
                              md_fp: str # file path of the feedback markdown file
                              ) -> github.Issue.Issue: # open issue
        "Create GitHub issue from markdown file."
        return self.ghg.create_issue_from_md(repo, md_fp)

    def fetch_issue(self,
                    repo:github.Repository.Repository, # target repository to fetch issue
                    component:str, # the component of the project grading, let it be proposal/checkpoint/final. Need to match the issue's title
                    ) -> github.Issue.Issue:
        "Fetch the issue on GitHub repo and choose related one"
        for issue in list(repo.get_issues()):
            if component in issue.title:
                return issue
        raise ValueError(f"Issue related to {component} did not found.")

    def parse_score_from_issue(self,
                               repo:github.Repository.Repository, # target repository to create issue
                               component:str, # the component of the project grading, let it be proposal/checkpoint/final. Need to match the issue's title
                               ) -> int: # the fetched score of that component
        "parse score from the template issue"
        issue = self.fetch_issue(repo, component)
        body = issue.body
        score = 0
        for line in body.split("\n"):
            if "Score =" in line and "[comment]" not in line:
                score = literal_eval(line.split("=")[1])
                return score
        raise ValueError(f"Score Parse Error. please check the score format on github. \n"
                         f"Issue URL: {issue.url}")

    def update_canvas_score(self,
                            group_name:str, # target group name on canvas group
                            assignment_id, # assignment id of the related component
                            score:float, # score of that component
                            post=False, # whether to post score via api. for testing purposes
                            ):
        "Post score to canvas"
        members = self.cg.group_to_emails[group_name]
        self.cg.link_assignment(assignment_id)
        for member in members:
            student_id = self.cg.email_to_canvas_id[member]
            if post:
                self.cg.post_grade(
                    student_id=student_id,
                    grade=score,
                    text_comment=f"Group: {group_name}"
                )
            else:
                print(f"{bcolors.WARNING}Post Disable{bcolors.ENDC}")
                print(f"For student: {member}, the score is {score}")

    def grade_project(self,
                      repo:github.Repository.Repository, # target repository to grade
                      component:str, # the component of the project grading, let it be proposal/checkpoint/final. Need to match the issue's title
                      assignment_id:int, # assignment id that link to that component of the project
                      canvas_group_name=None, # mapping from GitHub repo name to Group name. If not specified, the repository name will be used.
                      post=False, # whether to post score via api. For testing purposes
                      ):
        "grade github project components"
        score = self.parse_score_from_issue(repo, component)
        if canvas_group_name is not None:
            group_name = canvas_group_name[repo.name]
        else:
            group_name = repo.name
        self.update_canvas_score(group_name, assignment_id, score, post)



In [ ]:
show_doc(Grading.create_issue_from_md)

In [ ]:
show_doc(Grading.fetch_issue)

In [ ]:
show_doc(Grading.parse_score_from_issue)

In [ ]:
show_doc(Grading.update_canvas_score)

In [ ]:
show_doc(Grading.grade_project)